<a href="https://colab.research.google.com/github/staerkjoe/ML_colab/blob/main/CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision transformers ftfy regex tqdm
!pip install accelerate

In [ ]:
from transformers import CLIPProcessor, CLIPModel

# Load pretrained CLIP (text + image encoders)
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

device = "cuda"
clip_model = clip_model.to(device)

In [4]:
from PIL import Image

text = ["a red apple", "a blue car"]
image = [Image.open("/content/drive/MyDrive/Colab Notebooks/NLP/MiniProject/Sample Pictures/redapple.jpg"), Image.open("/content/drive/MyDrive/Colab Notebooks/NLP/MiniProject/Sample Pictures/airplane.jpg")]

inputs = clip_processor(text=text, images=image, return_tensors="pt", padding=True).to(device)
outputs = clip_model(**inputs)
similarity = outputs.logits_per_text.softmax(dim=-1)

In [5]:
print(similarity)

tensor([[1.0000e+00, 5.9555e-08],
        [2.3025e-01, 7.6975e-01]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
